<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/Pandas/%D0%A0%D0%B0%D0%BD%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%B8%20%25%20%D0%B4%D0%B8%D0%BD%D0%B0%D0%BC%D0%B8%D0%BA%D0%B0/%D0%A0%D0%B0%D0%BD%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B8_%25_%D0%B4%D0%B8%D0%BD%D0%B0%D0%BC%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
coderun=pd.read_csv('itresume-coderun.csv', encoding='1251')
# Преобразование строки с датой в дату
coderun['created_at']=pd.to_datetime(coderun['created_at'], format="%Y-%m-%d %H:%M:%S.%f")
original_index=coderun.index
# Создание нового датафрейма с сортировкой по задаче и дате
coderun_sorted=coderun.sort_values(['problem_id', 'created_at'])
# Создание столбца с рангом - группировка по пользователю
coderun_sorted['rnk_correct']=coderun_sorted.groupby('user_id').cumcount() + 1
coderun=coderun_sorted.loc[original_index]
# Создание столбца с рангом - группировка по пользователю и дате
coderun['rnk_for_df']=coderun.sort_values(['user_id', 'problem_id', 'created_at']).groupby('user_id')['created_at'].rank(method='first').astype(int)
coderun['created_at']=coderun['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
# Создание результирующего датафрейма
df=coderun[coderun['rnk_for_df']>3].copy()
df=df.sort_values('rnk_for_df').reset_index(drop=True)
# Подсчет количества уникальных пользователей по каждому рангу
df=df.groupby('rnk_for_df', as_index=False).agg(amount=('user_id', 'nunique'))
df=df.rename(columns={'rnk_for_df': 'rnk'})
# Расчет динамики между текущим количеством и предыдущим
df['diff']=df['amount'].pct_change().fillna(0).round(2)
coderun['rnk']=coderun['rnk_correct']
coderun=coderun.drop(columns=['rnk_correct', 'rnk_for_df'])
df


,rnk,amount,diff
0,4,660,0.00
1,5,625,-0.05
2,6,595,-0.05
3,7,569,-0.04
4,8,543,-0.05
...,...,...,...
1458,1462,1,0.00
1459,1463,1,0.00
1460,1464,1,0.00
1461,1465,1,0.00
